In [1]:
import requests
import os
import json

# --- 1. 설정 ---
# 🚨 [필수] 1단계에서 발급받은 Pixabay API 'Key'를 입력하세요.
API_KEY = "52913925-73a10d5765a70445e0bd9d899"  # 여기에 Key를 붙여넣으세요.
# ---------------------

# 검색 및 저장 설정
search_keyword = "pad thai", "팟타이", "thai stir fried noodles", "ผัดไทย", "pad thai noodle"                 # ✅ 검색어 (한글, 영어 모두 가능)
save_directory = "./pixabay/pad thai"  # ✅ 저장 폴더
total_images_needed = 400            # ✅ 목표 이미지 개수
min_size = 256                       # ✅ 최소 픽셀 크기

# --- 2. 저장 폴더 생성 ---
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    print(f"'{save_directory}' 폴더를 생성했습니다.")
else:
    print(f"'{save_directory}' 폴더가 이미 존재합니다.")

# --- 3. API 호출 및 이미지 다운로드 ---
api_url = "https://pixabay.com/api/"

image_count = 0  # 현재까지 다운로드한 (필터링된) 총 이미지 개수
page_num = 1     # API 호출을 시작할 페이지 번호
per_page_count = 100 # 한 번에 100개씩 요청 (Pixabay 최대 200)

print(f"'{search_keyword}' 픽사베이 크롤링 시작 (최소 {min_size}x{min_size} 크기만 저장)")

# 목표한 300개를 채울 때까지 계속 페이지를 넘기며 검색
while image_count < total_images_needed:

    # Pixabay API 파라미터 설정
    params = {
        "key": "",
        "q": "pad thai",
        "image_type": "photo",     # 사진만 검색
        "safesearch": "true",      # 안전 검색 활성화
        "page": page_num,
        "per_page": per_page_count
    }

    print(f"--- API 호출 (page={page_num}) ---")

    try:
        # API 요청
        response = requests.get(api_url, params=params)
        response.raise_for_status()

        data = response.json()

        items = data.get("hits", []) # Pixabay는 'hits' 키에 결과가 들어있음
        total_hits = data.get("totalHits", 0) # 검색된 총 이미지 수

        if not items:
            print("검색 결과가 없습니다.")
            break

        # --- 4. 개별 이미지 필터링 및 다운로드 ---
        for item in items:
            if image_count >= total_images_needed:
                break

            # Pixabay는 원본 크기를 'imageWidth', 'imageHeight'로 제공
            img_width = item.get('imageWidth', 0)
            img_height = item.get('imageHeight', 0)

            # ✅ 핵심 필터링 로직 (min_size = 224로 적용됨)
            if img_width >= min_size and img_height >= min_size:

                # ✨ 'largeImageURL' (1280px) 또는 'fullHDURL' (1920px) 사용
                # 'imageURL'은 너무 큰 원본일 수 있음
                image_url = item.get('fullHDURL') or item.get('largeImageURL')

                if not image_url:
                    print(f"  [SKIP] 크기는 맞으나 ({img_width}x{img_height}) 유효한 URL이 없음")
                    continue

                image_count += 1

                # 파일 확장자 추출 (URL에서 .jpg, .png 등)
                file_extension = os.path.splitext(image_url)[1]
                if not file_extension: file_extension = ".jpg" # 확장자 없으면 .jpg

                filename = f"{image_count:04d}{file_extension}"
                filepath = os.path.join(save_directory, filename)

                # --- 💡💡💡 수정된 부분 (403 Forbidden 오류 해결) 💡💡💡 ---
                # 이미지 다운로드 시도 (urllib 대신 requests 사용 및 User-Agent 헤더 추가)
                try:
                    # 403 Forbidden 방지를 위해 브라우저 헤더 추가
                    img_headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }
                    img_response = requests.get(image_url, headers=img_headers, stream=True, timeout=10)
                    img_response.raise_for_status() # 오류가 있으면 예외 발생

                    # 파일 쓰기 (스트림 방식)
                    with open(filepath, 'wb') as f:
                        for chunk in img_response.iter_content(chunk_size=8192):
                            f.write(chunk)

                    print(f"  [{image_count}/{total_images_needed}] 저장 (크기: {img_width}x{img_height}) -> {filepath}")
                except Exception as e:
                    print(f"  [실패] 다운로드 실패: {image_url} (오류: {e})")
                    image_count -= 1
                # --- 💡💡💡 수정 완료 💡💡💡 ---
            else:
                # 224x224보다 작아서 건너뛰는 경우
                print(f"  [SKIP] 크기가 작습니다 ({img_width}x{img_height})")

        # --- 5. 다음 페이지 준비 또는 종료 ---
        total_pages_available = (total_hits + per_page_count - 1) // per_page_count
        if page_num >= total_pages_available:
            print("Pixabay가 제공하는 모든 검색 결과를 확인했습니다.")
            break

        page_num += 1 # 다음 페이지 호출 준비

    except requests.exceptions.HTTPError as err:
        print(f"API 오류 발생: {err}")
        print("!!! API 키가 올바른지 확인하세요.")
        break
    except Exception as e:
        print(f"요청 중 알 수 없는 오류 발생: {e}")
        break

print(f"\n총 {image_count}개의 이미지를 다운로드했습니다 (최소 {min_size}x{min_size}).")

'./pixabay/pad thai' 폴더를 생성했습니다.
'('pad thai', '팟타이', 'thai stir fried noodles', 'ผัดไทย', 'pad thai noodle')' 픽사베이 크롤링 시작 (최소 256x256 크기만 저장)
--- API 호출 (page=1) ---
API 오류 발생: 400 Client Error: Bad Request for url: https://pixabay.com/api/?key=&q=pad+thai&image_type=photo&safesearch=true&page=1&per_page=100
!!! API 키가 올바른지 확인하세요.

총 0개의 이미지를 다운로드했습니다 (최소 256x256).


In [ ]:
import requests
import os
import json

# --- 1. 설정 ---
# 🚨 [필수] 1단계에서 발급받은 Pixabay API 'Key'를 입력하세요.
API_KEY = ""  # 여기에 Key를 붙여넣으세요.
# ---------------------

# 검색 및 저장 설정
search_keyword = "pad thai"                 # ✅ 검색어 (한글, 영어 모두 가능)
save_directory = "./팟타이_픽사베이_224px"  # ✅ 저장 폴더
total_images_needed = 300            # ✅ 목표 이미지 개수
min_size = 224                       # ✅ 최소 픽셀 크기

# --- 2. 저장 폴더 생성 ---
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    print(f"'{save_directory}' 폴더를 생성했습니다.")
else:
    print(f"'{save_directory}' 폴더가 이미 존재합니다.")

# --- 3. API 호출 및 이미지 다운로드 ---
api_url = "https://pixabay.com/api/"

image_count = 0  # 현재까지 다운로드한 (필터링된) 총 이미지 개수
page_num = 1     # API 호출을 시작할 페이지 번호
per_page_count = 100 # 한 번에 100개씩 요청 (Pixabay 최대 200)

print(f"'{search_keyword}' 픽사베이 크롤링 시작 (최소 {min_size}x{min_size} 크기만 저장)")

# 목표한 300개를 채울 때까지 계속 페이지를 넘기며 검색
while image_count < total_images_needed:

    # Pixabay API 파라미터 설정
    params = {
        "key": "",
        "q": "pad thai",
        "image_type": "photo",     # 사진만 검색
        "safesearch": "true",      # 안전 검색 활성화
        "page": page_num,
        "per_page": per_page_count
    }

    print(f"--- API 호출 (page={page_num}) ---")

    try:
        # API 요청
        response = requests.get(api_url, params=params)
        response.raise_for_status()

        data = response.json()

        items = data.get("hits", []) # Pixabay는 'hits' 키에 결과가 들어있음
        total_hits = data.get("totalHits", 0) # 검색된 총 이미지 수

        if not items:
            print("검색 결과가 없습니다.")
            break

        # --- 4. 개별 이미지 필터링 및 다운로드 ---
        for item in items:
            if image_count >= total_images_needed:
                break

            # Pixabay는 원본 크기를 'imageWidth', 'imageHeight'로 제공
            img_width = item.get('imageWidth', 0)
            img_height = item.get('imageHeight', 0)

            # ✅ 핵심 필터링 로직 (min_size = 224로 적용됨)
            if img_width >= min_size and img_height >= min_size:

                # ✨ 'largeImageURL' (1280px) 또는 'fullHDURL' (1920px) 사용
                # 'imageURL'은 너무 큰 원본일 수 있음
                image_url = item.get('fullHDURL') or item.get('largeImageURL')

                if not image_url:
                    print(f"  [SKIP] 크기는 맞으나 ({img_width}x{img_height}) 유효한 URL이 없음")
                    continue

                image_count += 1

                # 파일 확장자 추출 (URL에서 .jpg, .png 등)
                file_extension = os.path.splitext(image_url)[1]
                if not file_extension: file_extension = ".jpg" # 확장자 없으면 .jpg

                filename = f"{image_count:04d}{file_extension}"
                filepath = os.path.join(save_directory, filename)

                # --- 💡💡💡 수정된 부분 (403 Forbidden 오류 해결) 💡💡💡 ---
                # 이미지 다운로드 시도 (urllib 대신 requests 사용 및 User-Agent 헤더 추가)
                try:
                    # 403 Forbidden 방지를 위해 브라우저 헤더 추가
                    img_headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }
                    img_response = requests.get(image_url, headers=img_headers, stream=True, timeout=10)
                    img_response.raise_for_status() # 오류가 있으면 예외 발생

                    # 파일 쓰기 (스트림 방식)
                    with open(filepath, 'wb') as f:
                        for chunk in img_response.iter_content(chunk_size=8192):
                            f.write(chunk)

                    print(f"  [{image_count}/{total_images_needed}] 저장 (크기: {img_width}x{img_height}) -> {filepath}")
                except Exception as e:
                    print(f"  [실패] 다운로드 실패: {image_url} (오류: {e})")
                    image_count -= 1
                # --- 💡💡💡 수정 완료 💡💡💡 ---
            else:
                # 224x224보다 작아서 건너뛰는 경우
                print(f"  [SKIP] 크기가 작습니다 ({img_width}x{img_height})")

        # --- 5. 다음 페이지 준비 또는 종료 ---
        total_pages_available = (total_hits + per_page_count - 1) // per_page_count
        if page_num >= total_pages_available:
            print("Pixabay가 제공하는 모든 검색 결과를 확인했습니다.")
            break

        page_num += 1 # 다음 페이지 호출 준비

    except requests.exceptions.HTTPError as err:
        print(f"API 오류 발생: {err}")
        print("!!! API 키가 올바른지 확인하세요.")
        break
    except Exception as e:
        print(f"요청 중 알 수 없는 오류 발생: {e}")
        break

print(f"\n총 {image_count}개의 이미지를 다운로드했습니다 (최소 {min_size}x{min_size}).")

